# 01: Gemini Google Search Grounding Basics

This notebook introduces Gemini's built-in Google Search grounding capability,
which allows the model to search the web and ground its responses in real-time information.

## Learning Objectives

- Understand how Google Search grounding works
- Use the `GeminiGroundingTool` to make grounded queries
- Extract and display source citations
- Compare grounded vs non-grounded responses

In [ ]:
# Setup: Load environment and configure rich console
from aieng.agent_evals import (
    create_console,
    display_comparison,
    display_response,
    display_source_table,
)
from dotenv import load_dotenv


console = create_console()
load_dotenv(verbose=True)

## 1. Understanding Google Search Grounding

When you enable Google Search grounding, Gemini can:

1. Analyze your query and determine if web search would help
2. Automatically generate and execute search queries
3. Process search results and synthesize information
4. Return a response grounded in the search results with citations

This is different from traditional RAG because:
- No need to pre-index documents
- Access to real-time information
- The model decides when and what to search

In [ ]:
from aieng.agent_evals.knowledge_agent import GeminiGroundingTool
from rich.panel import Panel


# Initialize the grounding tool
tool = GeminiGroundingTool()

console.print(
    Panel(
        f"[green]✓[/green] Grounding tool initialized\n"
        f"[cyan]Model:[/cyan] {tool.model}",
        title="🔧 Setup Complete",
        border_style="green",
    )
)

## 2. Making Your First Grounded Query

In [ ]:
# Ask a question that requires current information
query = "What is the current population of Tokyo?"

console.print(f"\n[cyan]📝 Query:[/cyan] {query}\n")

with console.status("[cyan]Searching...[/cyan]", spinner="dots"):
    response = tool.search(query)

display_response(response, console=console, title="Tokyo Population")

In [ ]:
# Display sources in a detailed table format
display_source_table(response, console=console)

## 3. Comparing Grounded vs Non-Grounded Responses

This is where grounding truly shines. We'll ask about Toronto's record single-day snowfall.

**Why this example works:**
- The record was set on **January 25, 2026** - after the model's training data cutoff
- Without grounding, the model can only guess based on historical data it was trained on
- With grounding, the model searches the web and finds the recent news about this event

This clearly demonstrates that grounding enables access to information the model couldn't possibly know from training alone.

In [ ]:
from aieng.agent_evals.knowledge_agent import KnowledgeAgentConfig
from google import genai


config = KnowledgeAgentConfig()
client = genai.Client(api_key=config.openai_api_key)

# This question requires very recent information (Jan 2026)
# The non-grounded model will fail since its training data doesn't include this event
question = "Which day had the highest recorded snowfall in a single day in Toronto?"
expected_answer = "January 25, 2026"

console.print(f"\n[bold]Question:[/bold] {question}")
console.print(f"[dim]Expected Answer: {expected_answer}[/dim]\n")

# Without grounding - model relies on training data (cutoff before Jan 2026)
console.print("[dim]Generating without grounding...[/dim]")
response_no_grounding = client.models.generate_content(
    model=config.default_worker_model,
    contents=question,
)

# With grounding - model can search for current information
console.print("[dim]Generating with grounding...[/dim]")
response_grounded = tool.search(question)

# Side-by-side comparison using our display utility
display_comparison(response_no_grounding.text, response_grounded, console=console)

# Check if the grounded response contains the correct answer
if (
    expected_answer.lower() in response_grounded.text.lower()
    or "january 25" in response_grounded.text.lower()
):
    console.print("\n[green]✓ Grounded response contains the correct answer![/green]")
else:
    console.print("\n[yellow]⚠ Check the grounded response for accuracy[/yellow]")

## 4. Exercise: Try Your Own Queries

Try asking questions that:
- Require recent information (news, events, statistics)
- Need multiple facts combined
- Are about specific domains (sports, science, politics)

In [ ]:
# Try your own query
my_query = "What are the latest developments in fusion energy?"

console.print(f"[bold cyan]🔍 Query:[/bold cyan] {my_query}\n")

with console.status("[cyan]Searching the web...[/cyan]", spinner="dots"):
    my_response = tool.search(my_query)

display_response(my_response, console=console, title="Fusion Energy Developments")

## Summary

In this notebook, you learned:

1. How Google Search grounding works with Gemini
2. How to use the `GeminiGroundingTool` for grounded queries
3. How to extract and display source citations
4. The difference between grounded and non-grounded responses

**Next**: In the next notebook, we'll build a full knowledge agent that uses this grounding capability.

In [ ]:
console.print(
    Panel(
        "[green]✓[/green] Notebook complete!\n\n"
        "[cyan]Next:[/cyan] Open [bold]02_agent_basics.ipynb[/bold] to learn about the Knowledge Agent.",
        title="🎉 Done",
        border_style="green",
    )
)